In [5]:
import pandas as pd
import numpy as np

In [6]:
# Google
g_metadata_sitio = "datasets/google/metadata_sitio.parquet"
g_review = "datasets/google/g_review.parquet"

# Yelp
y_checkin = "datasets/yelp/checkin.parquet"
y_review = "datasets/yelp/review.parquet"
y_tip = "datasets/yelp/tip.parquet"
y_user = "datasets/yelp/user.parquet"

# Carga de Datos

**Google Maps**

*Metadata_sitio*

In [7]:
df_g_metadata_sitio = pd.read_parquet(g_metadata_sitio)

In [8]:
df_g_metadata_sitio.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,...,MISC.Amenities,MISC.Atmosphere,MISC.Crowd,MISC.From the business,MISC.Popular for,MISC.Dining options,MISC.Getting here,MISC.Recycling,MISC.Activities,MISC.Lodging options
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,...,None,None,None,None,None,None,None,None,None,None
1,"Elite Exteriors Roofing & Restoration, Inc.",None,0x87ccd3297badbb3b:0x18e80a5c9de19f16,None,45.597481,-127.269699,"[Roofing contractor, Construction company, Ser...",4.2,18,None,...,None,None,None,None,None,None,None,None,None,None
2,Harper's Plumbing,"Harper's Plumbing, 400 S Front St, Gurdon, AR ...",0x86330338d4921e25:0x4af02600facc4465,None,33.918058,-93.155579,[Plumber],5.0,1,None,...,None,None,None,None,None,None,None,None,None,None
3,Wylie Glass and Salvage Inc,"Wylie Glass and Salvage Inc, 1501 W 3rd St, Ho...",0x86338365e22e037b:0xf2ec124da3ecdfe6,None,33.664719,-93.606783,"[Salvage yard, Auto glass shop, Auto parts sto...",4.3,16,None,...,None,None,None,None,None,None,None,None,None,None
4,Wholesale Beauty & Convenience,"Wholesale Beauty & Convenience, 1750 Pine St, ...",0x8632dfdb9faed6a3:0x506f0e73cb79c8f2,None,34.123242,-93.071387,"[Convenience store, Cosmetics store]",2.9,8,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
df_g_metadata_sitio.describe()

,latitude,longitude,avg_rating,num_of_reviews,MISC
count,275001.000000,275001.000000,275001.000000,275001.000000,0.0
mean,37.451010,-92.399975,4.304241,29.328250,NaN
std,5.409527,16.353387,0.735298,52.443278,NaN
min,-2.772509,-176.639943,1.000000,1.000000,NaN
25%,33.743525,-99.501594,4.000000,6.000000,NaN
50%,38.270721,-87.667406,4.500000,15.000000,NaN
75%,41.297443,-80.302483,4.900000,35.000000,NaN
max,71.292346,180.000000,5.000000,5878.000000,NaN


In [10]:
df_g_metadata_sitio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   name                    274998 non-null  object 
 1   address                 267221 non-null  object 
 2   gmap_id                 275001 non-null  object 
 3   description             17908 non-null   object 
 4   latitude                275001 non-null  float64
 5   longitude               275001 non-null  float64
 6   category                273318 non-null  object 
 7   avg_rating              275001 non-null  float64
 8   num_of_reviews          275001 non-null  int64  
 9   price                   18801 non-null   object 
 10  hours                   199371 non-null  object 
 11  state                   275001 non-null  object 
 12  relative_results        247128 non-null  object 
 13  url                     275001 non-null  object 
 14  MISC.Service options

*reviews por Provincia*

In [11]:
df_g_review = pd.read_parquet(g_review)

In [12]:
df_g_review.head()

,user_id,name,time,rating,text,pics,gmap_id,resp.time,resp.text,resp,state
0,108317460579141970085,cayla layman,1630760310327,5,Making West Memphis proud! Love this spot for ...,[{'url': ['https://lh5.googleusercontent.com/p...,0x87d57755954b200d:0x157d3f3bad3d37a4,1.631040e+12,Thank you very much for your kind words. 😊 ple...,NaN,California
1,102123313902332683437,thomas mcdaniel,1631038071292,1,Small burrito period high price. U dont get yo...,None,0x87d57755954b200d:0x157d3f3bad3d37a4,1.631040e+12,Sorry for your negative experience. No one has...,NaN,California
2,104067094003049131996,tony shelby,1627697592294,5,I love the food and the staff are very friendly,None,0x87d57755954b200d:0x157d3f3bad3d37a4,1.627707e+12,Thank you. Please hurry back,NaN,California
3,107109337185185980940,james williamson,1624394171528,3,It was a New Restaurant and I never had there ...,None,0x87d57755954b200d:0x157d3f3bad3d37a4,1.624395e+12,Hi Mr. Williamson\nThanks for stopping by the ...,NaN,California
4,114611735524979608325,Robert Hardin,1630615588505,5,Great food and service,None,0x87d57755954b200d:0x157d3f3bad3d37a4,1.630669e+12,Thank you for stopping in. Please come again s...,NaN,California


In [13]:
df_g_review.describe()

,time,rating,resp.time,resp
count,8.023881e+06,8.023881e+06,9.891630e+05,0.0
mean,1.552970e+12,4.306171e+00,1.568196e+12,NaN
std,4.137102e+10,1.097876e+00,3.621126e+10,NaN
min,6.626016e+11,1.000000e+00,1.296971e+12,NaN
25%,1.526575e+12,4.000000e+00,1.545258e+12,NaN
50%,1.555595e+12,5.000000e+00,1.569844e+12,NaN
75%,1.580934e+12,5.000000e+00,1.598473e+12,NaN
max,1.631178e+12,5.000000e+00,1.631120e+12,NaN


In [14]:
df_g_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8023881 entries, 0 to 8023880
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user_id    object 
 1   name       object 
 2   time       int64  
 3   rating     int64  
 4   text       object 
 5   pics       object 
 6   gmap_id    object 
 7   resp.time  float64
 8   resp.text  object 
 9   resp       float64
 10  state      object 
dtypes: float64(2), int64(2), object(7)
memory usage: 673.4+ MB


**Yelp**

*checkin*

In [15]:
df_y_checkin = pd.read_parquet(y_checkin)

In [16]:
df_y_checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [17]:
df_y_checkin.describe()

,business_id,date
count,131930,131930
unique,131930,131930
top,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
freq,1,1


In [18]:
df_y_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


*review*

In [19]:
df_y_review = pd.read_parquet(y_review)

In [20]:
df_y_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [21]:
df_y_review.describe()

,stars,useful,funny,cool
count,6.990280e+06,6.990280e+06,6.990280e+06,6.990280e+06
mean,3.748584e+00,1.184609e+00,3.265596e-01,4.986175e-01
std,1.478705e+00,3.253767e+00,1.688729e+00,2.172460e+00
min,1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,5.000000e+00,1.182000e+03,7.920000e+02,4.040000e+02


In [22]:
df_y_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    object 
 1   user_id      object 
 2   business_id  object 
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
dtypes: float64(1), int64(3), object(5)
memory usage: 480.0+ MB


*tip*

In [23]:
df_y_tip = pd.read_parquet(y_tip)

In [24]:
df_y_tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [25]:
df_y_tip.describe()

,compliment_count
count,908915.000000
mean,0.012525
std,0.120763
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,6.000000


In [26]:
df_y_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           908915 non-null  object
 1   business_id       908915 non-null  object
 2   text              908915 non-null  object
 3   date              908915 non-null  object
 4   compliment_count  908915 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


*user*

In [27]:
df_y_user = pd.read_parquet(y_user)

In [28]:
df_y_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [29]:
df_y_user.describe() 

,review_count,useful,funny,cool,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
count,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06,2.105597e+06
mean,2.846875e+01,5.516853e+01,2.267667e+01,3.143013e+01,1.913746e+00,3.641863e+00,2.444596e+00,3.968618e-01,2.532631e-01,1.858665e-01,9.631663e-02,1.921464e+00,4.124809e+00,3.805567e+00,3.805567e+00,1.459133e+00,1.499514e+00
std,1.042146e+02,7.388068e+02,4.628811e+02,6.379274e+02,2.436510e+01,1.159519e+00,7.963704e+01,1.405658e+01,1.767698e+01,1.170051e+01,1.049796e+01,6.177753e+01,1.364953e+02,1.057484e+02,1.057484e+02,3.676692e+01,9.607754e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.880000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.900000e+01,1.500000e+01,3.000000e+00,4.000000e+00,0.000000e+00,4.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.747300e+04,2.062960e+05,1.858230e+05,1.998780e+05,1.249700e+04,5.000000e+00,2.578400e+04,1.350100e+04,1.418000e+04,1.365400e+04,1.266900e+04,5.903100e+04,1.010970e+05,4.996700e+04,4.996700e+04,1.593400e+04,8.263000e+04


In [30]:
df_y_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB
